In [1]:
import requests
import json
import time
from datetime import date
from datetime import datetime
from helpers.history_fetcher import HistoryFetcher
import dateutil.parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gdelt # pip install gdelt
import folium
import os
from tqdm import tqdm_notebook # conda install tqdm
import operator
from helpers.wiki_helpers import get_stability_for_country, make_folium_map, wiki_change_factor

import warnings
warnings.filterwarnings('ignore')

import itertools

%load_ext autoreload
%autoreload 2

pd.options.mode.chained_assignment = None

---
### Introduction

Our project idea came from the observation that some wikipedia pages seems to get updated a lot after a related events happens. This could be observed during the Catalonia crisis that happened in september-october 2017, Ukraine War in Donbass and many others. We then wondered how we could exploit this information and quickly agreed that those changes could be correlated to a country's stability. Therefore, the project aims to build a stability index for countries using wikipedia data.

#### Maps info
There are folium maps in this notebook. If youre viewing in on Github, open it here to see the maps:<br>
https://nbviewer.jupyter.org/github/Amaumaury/ada-2017/blob/master/project/Milestone2.ipynb


#### Part 1

The first step for our project was then to find evidence of the correlation between events and edits on wikipedia. To tackle this problem we decided to take a special case: the 2014 Ukraine war. Our approach is to show a visual argument (plot) that confirms our intuition.

#### Part 2

Knowing that the causility between events and wikipedia edits existed, we then decided to make a first prototype of our stability index. 

#### Part 3

In this part, we showed the limit of our approach. This let us make a plan for the next milestone

#### Part 4

This part exposes our plan for the next milestone.




---
### Part I - Finding evidence of the implication: main events => lots of wikipedia edits

#### 1 - Find a dataset that contains events for the Ukraine war. 

Intialy we selected the UCDP GED dataset. Unfortunately, as this dataset contains only armed events, we were not able able to find a good visual argument for our intuition. We thus decided to switch to a richer dataset: GDELT events. In order to handle the data, we used a library called **gdeltPyR**.

In [ ]:
# Initialize our GDELT fetcher and fetch data
gd1 = gdelt.gdelt(version=1)
results = gd1.Search(['2014 February 01','2014 April 30'],table='events')

In [ ]:
# Select events related to Ukraine
ukraine_events = results[results['ActionGeo_CountryCode'] == 'UP']

# Properly format the date for each event
ukraine_events['SQLDATE'] = ukraine_events['SQLDATE'].map(lambda d: pd.Timestamp(str(d)))
ukraine_events = ukraine_events.set_index('SQLDATE')

# Filter out events that are out of our timeframe
ukraine_events = ukraine_events.loc[ukraine_events.index >= pd.Timestamp('20140201')]
ukraine_events = ukraine_events.loc[ukraine_events.index <= pd.Timestamp('20140430')]

# Final holder for our events
ukraine_events_dates = list(ukraine_events.index)

We decided to select only events that occured between 02.01.2014 and 04.30.2014 as this timeframe bounds the beginning of the war.

#### 2 - Grab the history (i.e. edits) of Ukraine's wikipedia page.

In order to grab the edits history for any wikipedia page, we used wikipedia's API and proxied it through our HistoryFetcher class. 

In [ ]:
# Fetches Ukraine's wikipedia page history
history_fetcher = HistoryFetcher('Ukraine')
response = history_fetcher.get_history('20140201000000', '20140430000000')

# Keeps only the date field for each edit
ukraine_edits_dates = list(map(lambda revision: revision['timestamp'], response))

#### 3 - Plot the number of edits and events over time

This task was done ploting simple histograms with matplotlib. Both curve are normalized for visibility purposes. 

In [ ]:
# Add historgram for the number of edits on ukraine's wikipedia page (normalized)
n_wiki, bins_wiki, patches_wiki = plt.hist(ukraine_edits_dates,\
                                           bins=79,\
                                           normed=True,\
                                           color='blue',\
                                           alpha=0.5,\
                                           label='# of edits on Ukraine page')

# Add histogram for the number of events (normalized)
n_event, bins_event, patches_event = plt.hist(ukraine_events_dates,\
                                              bins=79,\
                                              color='green',\
                                              normed=True,\
                                              alpha=0.5,\
                                              label='# events in Ukraine')

plt.xticks(rotation=45)
plt.xlabel('Time (in days)')
plt.ylabel('Normalized number of edits/events')
plt.legend(loc='upper right')
plt.show()

In [ ]:
print('Pearson coefficient for our two normalized series is:', np.corrcoef(n_wiki, n_event)[1, 0], '\nThis is a rather high value that emphasize our intuition.')

#### Conclusions

This initial investigation, by combining data from two sources, provides arguments towards the causality between events and edits on wikipedia. 

---
### Part II - A prototype for our countries' stability index

With this part we're trying to analyse the rate of wikipedia changes to estimate the stability of the country and its other factors. We believe this analysis is interesting and could draw very interesting conclusions and could also help to find events that causw wikipedia change.

#### Analyzing dynamism of Wikipedia changes

We designed a simple classifier function ( $wiki\_change\_factor(...)$ ) to show how many time wikipedia content varies in time fo a given wikipedia page. Our classifier is based on the mean of the wikipedia changes. If for some month there are number of changes that vary from the mean value too much (more than outlier_factor * mean), we mark this event as "out of normal behaviour" and at the end we sum these events and divide by the whole number of wiki changes for the period.

In [ ]:
inst_uk = wiki_change_factor('Ukraine', 2011, outlier_factor=3.4, plot_on=True)
print('Wikipedia Instability for Ukraine :', inst_uk)

In [ ]:


inst_nr = wiki_change_factor('Norway', 2011, outlier_factor=3.4, plot_on=True)
print('Wikipedia Instability for Norway :', inst_nr)

As we can see above, our classifier is able to detect major events in wikipedia changes and assign a numerical
value to the countries wikipedia stability.

#### Plans:
We're planning to test different classifiers based on derivation to detect different typed of peaks in the data in a different, more error-prone way.

<br>
#### Comparing European Countries
By plotting this coefficient values for different countries in Europe, we can plot a Map with different Wikipedia dynamism rate 

In [ ]:
countries = ['Ukraine','Poland','Spain', 'Netherlands','France','Germany','Switzerland','Belgium',
            'Portugal','Italy','Great Britain', 'Ireland', 'Norway', 'Sweden','Denmark',
            'Slovakia','Czech Republic','Austria','Hungary','Croatia','Romania','Greece','Serbia',
            'Macedonia','Albania','Estonia','Latvia','Finland','Lithuania','Russia','Bulgaria','Moldova',
            'Belarus','Turkey','Iceland']

codes = ['UA','PL','ES', 'NL','FR','DE','CH','BE', 'PT','IT','GB','IE','NO','SE','DK',
        'SK','CZ','AT','HU','HR','RO','GR','RS','MK','AL','EE','LV','FI','LT','RU','BG','MD',
        'BY','TR','IS']

wiki_change_values = {}

for i in tqdm_notebook(range(len(countries))):
    wiki_change_values[codes[i]] = wiki_change_factor(countries[i], 2011, outlier_factor = 3.4)

    
sorted_x = sorted(wiki_change_values.items(), key=lambda x: x[1])
values = np.zeros([len(sorted_x),1])
names = ['']*len(sorted_x)

for i in range(len(sorted_x)):
    names[i] = sorted_x[i][0]
    values[i] = sorted_x[i][1]
values= np.squeeze(values)

plt.rcParams["figure.figsize"] = (8,8)
plt.bar(range(len(sorted_x)), values, align='center')
plt.xticks(range(len(sorted_x)), names, rotation=45)
plt.ylabel('National Wikipedia Instability')
plt.title('Wikipedia instability by country (Europe)')
plt.show()

We could observe, that  as expected some very developed and stable countries have very stable wikipedia, and the less stable also less stable wikipedia. However we could observe some interesting facts for some nations:
- Ukraine has lots of instability due to war
- Germany or Croatia has a lot of instability in its wikipedia - to be discovered why, maybe there are some major events to be correlated
- The computed instability factor highly depends on computation method, therefore we should prove better that our method is valid.

#### Wikipedia Instability Map in Europe.
In this part we're showing on the map the previously computed values for Wikipedia Instabilities per country on tha map. 

In [ ]:
colors_eu = ['#ffe6d0', '#fcbba1', '#fc9272', '#fb6a4a', '#ef3b2c', '#cb181d', '#a50f15', '#89000d']
thresh_eu = [0.03      , 0.06      , 0.1      , 0.13     , 0.16     , 0.2     , 0.23     , 0.26]


def wiki_changes_colors_eu(state):
    """Maps Unemplpoyment value with colors."""
    try:
        val = wiki_change_values[state]
    except KeyError:
        return '#bdbdbd'
    
    if np.isnan(val):
        return '#bdbdbd'
    
    length = len(colors_eu)
    for i in range(0, length-1):
        if val < thresh_eu[i]:
            return colors_eu[i]
    return  colors_eu[length-1]


m = make_folium_map('data/europe2.topojson.json', 'objects.europe',
                    wiki_changes_colors_eu, 0.0, 0.259, colors_eu, 
                    [56,10], 3.5, '% Wiki page instability 2010-2017')

m

### Countries Analysis Worldwide

1. Load the % of people using internet for all countries database from the UN Data.
2. Load All number of people for all countries database from the UN Data.
3. Merge the 2 dataframes and compute the number of people using internet 
4. Load countries names in ISO-2 codes for the map

In [2]:
nr_internet_people = pd.read_csv('data/nr_people_internet.csv')
cols_to_keep = ['Country or Area','Value']
nr_internet_people = nr_internet_people[cols_to_keep]
nr_internet_people.columns=['Country','Internet Users%']
nr_internet_people.set_index('Country',inplace=True)

nr_all_people = pd.read_csv('data/nr_people_all.csv')
nr_all_people = nr_all_people[nr_all_people['Variant']=='Medium'][cols_to_keep]
cols_to_keep = ['Country or Area','Value']
nr_all_people = nr_all_people[cols_to_keep]
nr_all_people.columns=['Country','Population[k]']
nr_all_people.set_index('Country',inplace=True)


population_data = pd.concat([nr_all_people, nr_internet_people], axis=1, join='inner')
population_data['Internet users[k]'] = population_data['Population[k]'] * population_data['Internet Users%']/100
population_data.head(5)


countries_data = pd.read_csv('data/country-codes.csv')[1:]
cols_to_keep = ['official_name_en','ISO3166-1-Alpha-3','Least Developed Countries (LDC)','Region Name',
                'Small Island Developing States (SIDS)', 'is_independent']
countries_data = countries_data[cols_to_keep]

# selectbonly independent countires
#countries_data = countries_data[countries_data['is_independent']=='Yes']

#countries_data = countries_data[countries_data['Least Developed Countries (LDC)']!='x']
#countries_data = countries_data[countries_data['Small Island Developing States (SIDS)']!='x']

cols_to_keep = ['official_name_en','ISO3166-1-Alpha-3','Region Name']

countries_data = countries_data[cols_to_keep]
countries_data.columns = ['Country', 'Code','Region']
countries_data['Wiki Stability MLE'] = np.NaN;
countries_data['Wiki Instability old'] = np.NaN;
countries_data['Wiki mean change day'] = np.NaN;
countries_data.set_index('Country',inplace=True)


countries_data = pd.concat([countries_data, population_data], axis=1, join='inner')
countries_data['Big Internet'] = countries_data['Internet users[k]'] > 1000
countries_data.reset_index(inplace=True)

gdp_data = pd.read_csv('data/UNdata_GDP.csv')
keep_cols = ['Country or Area', 'Value']

gdp_data = gdp_data[keep_cols]
gdp_data.columns = ['Country', 'GDP']
gdp_data.set_index('Country', inplace=True)
#countries_data.reset_index(inplace=True)
countries_data.set_index('Country', inplace=True)
countries_data = pd.concat([countries_data, gdp_data], axis=1, join='inner')
countries_data.reset_index(inplace=True)

countries_data['GDP per capita'] = countries_data['GDP'] / countries_data['Population[k]']

countries_data.head()

,Country,Code,Region,Wiki Stability MLE,Wiki Instability old,Wiki mean change day,Population[k],Internet Users%,Internet users[k],Big Internet,GDP,GDP per capita
0,Afghanistan,AFG,Asia,NaN,NaN,NaN,35530.081,8.260000,2934.784691,True,623.184798,0.017540
1,Albania,ALB,Europe,NaN,NaN,NaN,2930.187,63.252933,1853.429211,True,3984.234302,1.359720
2,Algeria,DZA,Africa,NaN,NaN,NaN,41318.142,38.200000,15783.530244,True,4154.118319,0.100540
3,Andorra,AND,Europe,NaN,NaN,NaN,76.965,96.910000,74.586782,False,39896.376936,518.370388
4,Angola,AGO,Africa,NaN,NaN,NaN,29784.193,12.400000,3693.239932,True,4714.065956,0.158274


In [3]:

# adding the countires that are missing in the map
import json

geo_data = json.load(open('data/countries.geo.json'))

keys=[k['id'] for k in geo_data['features']]
missing_keys=set(keys)-set(countries_data['Code'])

for k in (missing_keys):
    temp = pd.DataFrame([['', k,'',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN, np.NaN]], columns=countries_data.columns)
    countries_data = countries_data.append(temp, ignore_index=True)

countries_data.head(5)


,Country,Code,Region,Wiki Stability MLE,Wiki Instability old,Wiki mean change day,Population[k],Internet Users%,Internet users[k],Big Internet,GDP,GDP per capita
0,Afghanistan,AFG,Asia,NaN,NaN,NaN,35530.081,8.260000,2934.784691,1.0,623.184798,0.017540
1,Albania,ALB,Europe,NaN,NaN,NaN,2930.187,63.252933,1853.429211,1.0,3984.234302,1.359720
2,Algeria,DZA,Africa,NaN,NaN,NaN,41318.142,38.200000,15783.530244,1.0,4154.118319,0.100540
3,Andorra,AND,Europe,NaN,NaN,NaN,76.965,96.910000,74.586782,0.0,39896.376936,518.370388
4,Angola,AGO,Africa,NaN,NaN,NaN,29784.193,12.400000,3693.239932,1.0,4714.065956,0.158274


In [ ]:
#countries_data = pd.read_pickle('countries_data.pkl')
changes_europe = countries_data[countries_data['Region'] == 'Europe']['Wiki mean chang mo'].values
people_europe = countries_data[countries_data['Region'] == 'Europe']['Internet users[k]'].values
changes_asia = countries_data[countries_data['Region'] == 'Asia']['Wiki mean chang mo'].values
people_asia = countries_data[countries_data['Region'] == 'Asia']['Internet users[k]'].values
changes_africa = countries_data[countries_data['Region'] == 'Africa']['Wiki mean chang mo'].values
people_africa = countries_data[countries_data['Region'] == 'Africa']['Internet users[k]'].values

changes_americas = countries_data[countries_data['Region'] == 'Americas']['Wiki mean chang mo'].values
people_americas = countries_data[countries_data['Region'] == 'Americas']['Internet users[k]'].values

changes_oceania = countries_data[countries_data['Region'] == 'Oceania']['Wiki mean chang mo'].values
people_oceania = countries_data[countries_data['Region'] == 'Oceania']['Internet users[k]'].values

plt.semilogy(changes_europe,people_europe,'+r')
plt.hold(True)
plt.semilogy(changes_asia,people_asia,'+b')
plt.semilogy(changes_africa,people_africa,'+g')
plt.semilogy(changes_americas,people_americas,'+m')
plt.semilogy(changes_oceania,people_oceania,'+y')
plt.legend(['Europe','Asia','Africa','Americas','Oceania'])
plt.title('Log scale (only Y)')
plt.xlabel('Wiki mean chang mo')
plt.ylabel('Internet users[k]')
plt.show()

In [ ]:
intervals =  np.append(0, np.append([ 1000.0, 10000, 100000], np.inf))
colors =     ['+r',  '+g',   '+b'  ,'+m'  ,'+y' , '+c',   '+k'];



for i in range(len(intervals)-1):
    intervals[i], intervals[i+1]
    colors[i]
    
    changes = countries_data[(countries_data['GDP'] > intervals[i]) & (countries_data['GDP'] <= intervals[i+1])]['Wiki mean chang mo'].values
    people = countries_data[(countries_data['GDP'] > intervals[i]) & (countries_data['GDP'] <= intervals[i+1])]['Internet users[k]'].values

    plt.semilogy(changes,people,colors[i])
    plt.hold(True)

plt.legend(intervals.astype(str)[:-1])
plt.title('GDP of the country')
plt.xlabel('Wiki mean chang mo')
plt.ylabel('Internet users[k]')
plt.show()


In [ ]:
intervals =  np.append(0, np.append([ 2, 7, 20], np.inf))
colors =     ['+r',  '+g',   '+b'  ,'+m'  ,'+y' , '+c',   '+k'];



for i in range(len(intervals)-1):
    intervals[i], intervals[i+1]
    colors[i]
    
    changes = countries_data2[(countries_data2['GDP per capita'] > intervals[i]) & (countries_data2['GDP per capita'] <= intervals[i+1])]['Wiki mean chang mo'].values
    people = countries_data2[(countries_data2['GDP per capita'] > intervals[i]) & (countries_data2['GDP per capita'] <= intervals[i+1])]['Internet users[k]'].values

    plt.semilogy(changes,people,colors[i])
    plt.hold(True)

plt.legend(intervals.astype(str)[:-1])
plt.title(' > GDP per capita')
plt.xlabel('Wiki mean chang mo')
plt.ylabel('Internet users[k]')
plt.show()



In [28]:
countries_data[163:164]

,Country,Code,Region,Wiki Stability MLE,Wiki Instability old,Wiki mean change day,Population[k],Internet Users%,Internet users[k],Big Internet,GDP,GDP per capita
163,United States,USA,Americas,NaN,0.181298,0.747875,324459.463,74.55,241884.529666,1.0,56053.841155,0.172761


#### Analyse the instability and mean changes in wikipedia for all the countries 
1. Perform the internet-data-fetching and compute the instability factor and the mean changes.
2. Compute the ratio of people using internet and the mean changes in wikipedia for each country (contribution ratio)
3. Check if the internet is not too small - for very small cuntries the values are probably wrong
4. Cmpute the logarightm of 2. (because the values are in range 0.3 to 30 )

In [ ]:
countries_data = pd.read_pickle('countries_data.pckl')
countries = countries_data['Country'].values

countries[countries == 'Syrian Arab Republic'] = 'Syria'
countries[163] = 'United States'
countries[countries == 'Unites States of America'] = 'United States'

countries_data[countries_data['Country']==countries[i] ]['Unites States of America'] = np.NaN

codes = countries_data['Code'].values

for i in tqdm_notebook(range(len(countries))):
    # if not already fetched
    
    start = '20130101000000'
    stop = '20171101000000'
    try:
        if((np.isnan(countries_data[countries_data['Country']==countries[i] ]['Wiki Stability MLE'].values[0]))):
            print('fetching',countries[i])
            stab_factor, mean = get_stability_for_country( countries[i], start, stop)
            instab_factor = wiki_change_factor( countries[i], 2013, 2017, outlier_factor=3.4) 
            print(countries[i],'stab:',stab_factor,'instab',instab_factor,'mean', mean)
            countries_data[i-1:i]['Wiki Stability MLE'] = stab_factor
            countries_data[i-1:i]['Wiki Instability old'] = instab_factor
            countries_data[i-1:i]['Wiki mean change day'] = mean
            # save the values every time
            pd.to_pickle(countries_data, 'countries_data.pckl')
            
        else:
            print(countries[i],'Done already')
    except KeyError:
        break;
    except:
        print('skipped: ',countries[i], e)


A Jupyter Widget

Afghanistan Done already
Albania Done already
Algeria Done already
Andorra Done already
Angola Done already
Antigua and Barbuda Done already
Argentina Done already
Armenia Done already
Aruba Done already
Australia Done already
Austria Done already
Azerbaijan Done already
Bahrain Done already
Bangladesh Done already
Barbados Done already
Belarus Done already
Belgium Done already
Belize Done already
Benin Done already
Bermuda Done already
Bhutan Done already
Bosnia and Herzegovina Done already
Botswana Done already
Brazil Done already
Brunei Darussalam Done already
Bulgaria Done already
Burkina Faso Done already
Burundi Done already
Cambodia Done already
Cameroon Done already
Canada Done already
Cayman Islands Done already
Central African Republic Done already
Chad Done already
Chile Done already
Colombia Done already
Comoros Done already
Costa Rica Done already
Croatia Done already
Cuba Done already
Cyprus Done already
Denmark Done already
Djibouti Done already
Dominica Done already
Dom

In [ ]:
countries_data['Internet users[k]'] = countries_data['Population[k]'] * countries_data['Internet Users%']/100
countries_data['Big Internet'] = (countries_data['Internet users[k]'] > 2000)
countries_data_relevant = countries_data
countries_data_relevant.head()

In [ ]:
countries_data_relevant['Wiki contrib ratio'] = 1000*countries_data_relevant['Wiki mean chang mo'] / countries_data_relevant['Internet users[k]'].values
countries_data_relevant['Wiki contrib norm'] = countries_data_relevant['Wiki contrib ratio'] * (countries_data_relevant['Big Internet'])
countries_data_relevant['Wiki contrib log'] = np.log10(countries_data_relevant['Wiki contrib norm'])
countries_data_relevant.fillna(-3.0, inplace=True)
countries_data_relevant.replace([np.inf, -np.inf], -3.0, inplace=True)
countries_data_relevant.to_pickle('countries_data_relevant.pkl')
countries_data_relevant.sort_values(['Wiki contrib norm'], ascending=0)[70:]

### Make a folium map of the countries wiki contribution
A ratio between mean wikipedia changes and people using internet   (the higher, the better)

In [ ]:

m = folium.Map(location=[30, -30],
                   tiles='Mapbox Bright', zoom_start=2.5)

m.geo_json(geo_path=r'data/countries.geo.json', data=countries_data,
             columns=['Code', 'Wiki contrib log'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.5, line_opacity=0.1,
             legend_name='Data Rate ')


m

#### Conclusions to Part II
1. We can see that usually smaller countries has more stable wikipedia --> less things are happening
2. We can observe that in Ukraine there were major changes, probably due to War
3. We can observe that in Spain there were major changes, probably due to Catalonia protest
3. High changes in Germany, Croatia or Lithuania are bizzare, this will be examined in further analysis with events
4. Greece is not being flaged by our index. This is not normal considering the difficulties the country encoutered in 2011

#### Plans
Make the same map for all countries around the world, not only for Europe. 

---
### Part III - Showing the limits

At the moment, our index only considers the main wikipedia page of a country. This is clearly a limitation as information is very coarse grain at this level. Indeed, we are loosing a lot of information on some smaller events that might be interesting to consider like for example the Paris terrorists attack in 2015.

In [ ]:
# Fetches France's wikipedia page history
france_history_fetcher = HistoryFetcher('France')
france_history = france_history_fetcher.get_history('20150101000000', '20160101000000')

france_edits_dates = list(map(lambda revision: revision['timestamp'], france_history))

In [ ]:
# Add historgram for the number of edits on Frances's wikipedia page
n_wiki, bins_wiki, patches_wiki = plt.hist(france_edits_dates,\
                                           bins=52,\
                                           color='blue',\
                                           alpha=0.5,\
                                           label='# of edits on France page')

# Add the two main terrorists attacks that happened in 2015 on the timeline
plt.scatter([dateutil.parser.parse('20150107'), dateutil.parser.parse('20151113')], [0, 0], label='Terrorist attacks')

plt.xticks(rotation=45)
plt.xlabel('Time (in weeks)')
plt.ylabel('# of edits')
plt.legend(loc='upper right')
plt.show()

First terror attack did not trigger any immediate change on the wikipedia page (remember that each bar correspond to one week). 

Second terror attack could have maybe increased the number of edits but the bump is too low and would be almost impossible to detect automaticaly.

This problem in the resolution of our information motivates one of our objectives for next milestone.

---
### Part IV - Plan for next milestone

As of part 1, we're planning to find more event like this or maybe even semi-automate the process, to check if some event had some effect on wikipedia change rate.

Our priority for next milestone will be to improve our event detector that was exposed in part 2. We will also improve the formula that computes the stability index.

Depending on the time left we will try to improve our countries description using multiple wikipedia pages instead of using only the main one. This will let us have a much better view on the impact of events on any country. 